In [31]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [32]:
from proj1_helpers import *
from data_helpers import *
from cross_validation import *
from plots import *

In [33]:
#LOAD TRAINING DATA
DATA_TRAIN_PATH = 'data/train.csv'
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

In [34]:
#CLEAN TRAINING DATA
bounds = [0.0, 0.5]
tx_clean, ind_remov = treat_undefined_values(bounds, tX)
ind_remov

[4, 5, 6, 12, 26, 27, 28]

In [35]:
#MODEL BUILDING
tx, mean, std = standardize(tx_clean,0)
tx_augmented = build_poly_all_features(tx,3)
y,tx = build_model_data(tx_augmented,y)
y = classify(y)
num_samples = len(y)
num_features = tx.shape[1]
y = y.reshape(num_samples,1)


In [36]:
num_samples, num_features
tx.shape

(250000, 70)

In [37]:
from compute_gradient import *
from cost import *
from implementations import *

In [38]:
#Initial weight


#Maximum iterations through the whole set
max_iter = 100

#0.001 nan
#0.0005 nan

#Step-size
gammas = np.logspace(-6,-1, 7)
lambdas = np.logspace (-5,-1,7)


#Regularization factor
# lambda_ = 0.01 142000 loss 10 000 iters
#lambda_ = 0.001 141000 loss 10 000 iters
#lambda_ = 0.02
#loss_min = np.inf
tx.shape,y.shape,y

((250000, 70), (250000, 1), array([[1.],
        [0.],
        [0.],
        ...,
        [1.],
        [0.],
        [0.]]))

In [ ]:
losses_tr = np.zeros((len(gammas),len(lambdas)))
losses_val = np.zeros((len(gammas),len(lambdas)))

k_fold = 2
seed = 1
loss_min = np.inf
initial_w = np.zeros((tx.shape[1],1))

#CHOOSE LAMBDA AND GAMMA
for ind_lambda, lambda_ in enumerate(lambdas):
    for ind_gamma ,gamma_ in enumerate (gammas):
        loss_training = 0
        loss_validation = 0
        training_accuracy = 0
        validation_accuracy = 0
        for i,k in enumerate(range(k_fold)):
            
            tx_train, y_train, tx_val, y_val = cross_validation(y, tx, k, k_fold, seed)
            w, loss_train_k = reg_logistic_regression(y_train, tx_train, lambda_, initial_w, max_iter, gamma_)
            loss_val_k = reg_logistic_loss (y_val,tx_val,w,lambda_)/len(y_val)
            loss_train_k /= len(y_train)
            loss_training += loss_train_k
            loss_validation += loss_val_k
        
        
        training_accuracy /= k_fold
        validation_accuracy /= k_fold
        loss_training /= k_fold
        loss_validation /= k_fold
        print(loss_training)
        print(loss_validation)
        print(training_accuracy)
        print(validation_accuracy)
        if (loss_validation < loss_min):
            weights_star = w
            lambda_star = lambda_
            gamma_star = gamma_
            loss_min = loss_validation
        
        losses_tr[ind_lambda][ind_gamma] = loss_training
        losses_val[ind_lambda][ind_gamma] = loss_validation



/Users/Sofia/Desktop/ML_Project1/cost.py:20: RuntimeWarning: divide by zero encountered in log
  loss = y.T.dot(np.log(pred)) + (1 - y).T.dot(np.log(1 - pred))


nan
nan
0.0
0.0
nan
nan
0.0
0.0
nan
nan
0.0
0.0


In [26]:
lambdas

array([1.00000000e-05, 4.64158883e-05, 2.15443469e-04, 1.00000000e-03,
       4.64158883e-03, 2.15443469e-02, 1.00000000e-01])

In [27]:
gammas

array([1.00000000e-06, 6.81292069e-06, 4.64158883e-05, 3.16227766e-04,
       2.15443469e-03, 1.46779927e-02, 1.00000000e-01])

In [28]:
losses_tr, losses_val

(array([[0.69319692,        inf,        nan,        nan,        nan,
                nan,        nan],
        [       nan,        nan,        nan,        nan,        nan,
                nan,        nan],
        [       nan,        nan,        nan,        nan,        nan,
                nan,        nan],
        [       nan,        nan,        nan,        nan,        nan,
                nan,        nan],
        [       nan,        nan,        nan,        nan,        nan,
                nan,        nan],
        [       nan,        nan,        nan,        nan,        nan,
                nan,        nan],
        [       nan,        nan,        nan,        nan,        nan,
                nan,        nan]]),
 array([[0.69288124, 0.69210063,        nan,        nan,        nan,
                nan,        nan],
        [       nan,        nan,        nan,        nan,        nan,
                nan,        nan],
        [       nan,        nan,        nan,        nan,        nan,
  

In [29]:
loss_min, lambda_star, gamma_star

(0.6921006315995246, 1e-05, 6.8129206905796085e-06)

In [30]:
#PREDICT TRAINING ACCURACY
y_pred_training = predict_labels(weights_star, tx,'logistic')
count = 0
for i in range(num_samples):
    if (y_pred_training[i] == -1 and y[i] == 0):
        count += 1
    elif (y_pred_training[i] == y[i]):
        count += 1
accuracy_training = (count *100) / num_samples

accuracy

61.3084

In [198]:
#LOAD TEST SET
DATA_TEST_PATH = 'data/test.csv'
ytest, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [203]:
#CLEAN AND STANDARDIZE TEST SET
tX_test_clean = np.delete(tX_test, ind_remov, axis=1)
tX_test_clean.shape


(568238, 23)

In [206]:
tx_test_clean = remove_undefined_values (tX_test_clean)
tx_test_clean.shape

(568238, 23)

In [207]:
tx_test_clean, _, _ = standardize(tx_test_clean,0)


In [210]:
tx_test_augmented = build_poly_all_features(tx_test_clean,3)

(568238, 23)

In [213]:
tx_test_augmented.shape

(568238, 46)

In [214]:
#BUILD TEST MODEL
ytest, tx_test = build_model_data(tx_test_augmented,ytest)

In [215]:
#PREDICT LABELS
OUTPUT_PATH = 'data/submission_reg_logistic-degre3.csv'
y_pred = predict_labels(weights_star, tx_test,'logistic')
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)